In [43]:
import numpy as np
import pandas as pd
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

In [62]:
file_path = 'r29i_os_73.sav'
df, meta = pyreadstat.read_sav(file_path)
df = df[['yj13.2','yj72.18a','yj6.2','yj10.2','yj21b', 'yj1.1.1']]

df.rename(columns = {'yj13.2':'ЗП',
                     'yj72.18a':'Образование',
                     'yj6.2':'РабочаяНеделя',
                     'yj10.2':'Премия',
                     'yj21b':'Отпуск',
                     'yj1.1.1': 'Удовлетворенность'},
                inplace = True)

# Новые датасеты с удаленными шумами и выбросами, пропуски заменены на среднее и медиану соответственно
data_mean = df.copy()   
data_median = df.copy()  

df

,ЗП,Образование,РабочаяНеделя,Премия,Отпуск,Удовлетворенность
0,NaN,4.0,NaN,NaN,NaN,NaN
1,NaN,1.0,NaN,NaN,NaN,NaN
2,NaN,6.0,NaN,NaN,NaN,NaN
3,NaN,2.0,NaN,NaN,NaN,NaN
4,25000.0,5.0,40.0,99999996.0,42.0,3.0
...,...,...,...,...,...,...
12115,NaN,10.0,NaN,NaN,NaN,NaN
12116,40000.0,6.0,50.0,99999996.0,14.0,2.0
12117,99999997.0,6.0,48.0,NaN,58.0,2.0
12118,NaN,NaN,NaN,NaN,NaN,NaN


### Обнаружение выбросов

In [63]:
def detect_deviation(data: pd.Series) -> tuple[float, float]:
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1

    mx = Q3 + 1.5 * IQR
    mn = Q1 - 1.5 * IQR

    # return tuple(map(float, (mn.loc[0], mx.loc[0])))
    return mn, mx

dev = {}

In [ ]:
for column in df.columns:
    data = df[column]

    # Удаляем очевидные шумы
    data[data >= 99999996] = np.nan
    if column == 'РабочаяНеделя':
        data[data == 168] = np.nan
    if column == 'ЗП':
        data[data == 0] = np.nan
    if column == 'Образование':
        data[data > 17] = np.nan

    dev[column] = detect_deviation(data) # Находим выбросы (границы - min и max возможные значения)

pprint(dev)

{'ЗП': (np.float64(-12000.0), np.float64(68000.0)),
 'Образование': (np.float64(-10.0), np.float64(22.0)),
 'Отпуск': (np.float64(7.5), np.float64(51.5)),
 'Премия': (np.float64(-7500.0), np.float64(20500.0)),
 'РабочаяНеделя': (np.float64(28.0), np.float64(60.0)),
 'Удовлетворенность': (np.float64(0.5), np.float64(4.5))}


### Составляем новые данные 

In [ ]:
def new_data(data, flag):
    # Удаляем пропуски в целевой переменной 
    data['Удовлетворенность'][data['Удовлетворенность'] >= 9.9999997e+07] = np.nan
    data = data.dropna(subset=['Удовлетворенность'])

    # Сокращаем количество классов образования до 4
    dict = {}
    for _ in range(17):
        i = _ + 1
        if (i == 1) or (i == 2):
            dict[i] = 1
        elif (i <= 6) or (i == 15):
            dict[i] = 2
        elif (i <= 12):
            dict[i] = 3
        else:
            dict[i] = 4

    data['Образование'] = data['Образование'].replace(dict)
    

    

In [ ]:
new_data = df.copy()



new_data = new_data.drop(new_data[(new_data['ЗП'] <= 0) 
                | (new_data['ЗП'] > 68000)].index)
new_data['ЗП'] = new_data['ЗП'].replace(np.nan, new_data['ЗП'].median())

new_data['Премия'] = new_data['Премия'].replace(99999996, np.nan)
new_data = new_data.drop(new_data[(new_data['Премия'] < 0) 
                | (new_data['Премия'] > 20500)].index)
new_data['Премия'] = new_data['Премия'].replace(np.nan, new_data['Премия'].median())


new_data = new_data.drop(new_data[(new_data['Отпуск'] < 7.5) 
                | (new_data['Отпуск'] > 51.5)].index)
new_data['Отпуск'] = new_data['Отпуск'].replace(np.nan, new_data['Отпуск'].median())


new_data = new_data.drop(new_data[(new_data['Рабочая неделя'] < 28) 
                | (new_data['Рабочая неделя'] > 60)].index)
new_data['Рабочая неделя'] = new_data['Рабочая неделя'].replace(np.nan, new_data['Рабочая неделя'].median())

new_data.loc[new_data['Образование'] > 17, 'Образование'] = np.nan
new_data['Образование'] = new_data['Образование'].replace(np.nan, new_data['Образование'].median())




new_data = new_data.rename(columns={'Рабочая неделя': 'РабочаяНеделя'})

new_data_median = new_data
pyreadstat.write_sav(new_data_median, 'data_median.sav')
new_data_median

,ЗП,Образование,РабочаяНеделя,Премия,Отпуск,Удовлетворенность
0,25000.0,2.0,40.0,5000.0,28.0,2.0
1,25000.0,1.0,40.0,5000.0,28.0,2.0
2,25000.0,2.0,40.0,5000.0,28.0,2.0
3,25000.0,1.0,40.0,5000.0,28.0,2.0
4,25000.0,2.0,40.0,5000.0,42.0,3.0
...,...,...,...,...,...,...
12114,25000.0,3.0,40.0,5000.0,28.0,2.0
12115,25000.0,3.0,40.0,5000.0,28.0,2.0
12116,40000.0,2.0,50.0,5000.0,14.0,2.0
12118,25000.0,2.0,40.0,5000.0,28.0,2.0
